<a href="https://colab.research.google.com/github/xRame/Box/blob/master/NLP_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source "train.parquet" and "test.parquet" had been recompressed 
with engine "fastparquet" and compression "gzip". 
Original "pyarrow" with "snappy" crashes Jupyter )(
"fastparquet" with "gzip" properlly works in Jupyter as well, 
as in Google Collab

# Prerequisites

In [ ]:
!pip install --upgrade numpy pandas matplotlib fastparquet nltk sklearn gensim

In [10]:
                                import re
                                import numpy
                                import pandas
from    matplotlib              import pyplot
                                import fastparquet
                                import nltk
from    nltk.corpus             import stopwords
from    nltk.stem               import WordNetLemmatizer
from    nltk.tokenize           import WordPunctTokenizer
from    sklearn.preprocessing   import OneHotEncoder
from    gensim.models           import Word2Vec

In [11]:
# Use this variable to suppress output
DevNull = 0

In [12]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
Path = "./"

In [14]:
def ReadParquet(FName):
    return pandas.read_parquet(FName, engine = 'fastparquet')

def SaveParquet(DFrame, FName):
    DFrame.to_parquet(FName, engine = 'fastparquet', compression = 'gzip')

# Reading source data

In [15]:
TestDF = ReadParquet(Path + 'test.parquet').reset_index()
TestDF.shape

(12000, 4)

In [50]:
TestDF.head()

,Id,Body,Tags,Title,Text,Stop,NumOfStopwords,NumOfTags,NumOfWords,NumOfLetters
0,46785369,I was given a decision tree with sample data i...,<math><machine-learning><artificial-intelligen...,Desecion trees ended up with same given tree a...,"[given, decision, tree, sample, data, class, s...","[i, was, a, with, in, to, after, the, with, th...",29,5,37,233
1,38908667,<p>I am moving a test application over to Xcod...,<ios><swift><swift3><xcode8><nserror>,Value of type 'Error' has no member 'code',"[moving, test, application, xcode, beta, conve...","[p, i, am, a, over, to, and, my, to, i, am, wi...",37,5,56,291
2,44703012,<p>I would like to be able to perform linting ...,<jenkins><jenkins-pipeline>,How do I lint Jenkins pipelines from the comma...,"[would, like, able, perform, linting, jenkins,...","[p, i, to, be, to, on, and, it, that, is, not,...",24,2,19,118
3,34910435,"<p>I’m using <a href=""https://github.com/ftlab...",<javascript><css><mobile-safari><fastclick.js>...,Is there a way to prevent fastclick from firin...,"[using, href, http, github, com, ftlabs, fastc...","[p, i, m, a, a, on, a, with, because, i, to, t...",66,5,67,382
4,51632121,I have some 5 Test cases and I want them to ke...,<android><testing><appium><katalon-studio>,How to loop test cases in katalon Studio?,"[test, case, want, keep, running, time, order,...","[i, have, some, and, i, them, to, for, in, how...",22,4,24,131


In [16]:
TrainDF = ReadParquet(Path + 'train.parquet').reset_index()
TrainDF.shape

(48000, 5)

In [19]:
TrainDF.head()

,Id,Body,Tags,Title,target
0,34862324,Algorithmm:\r\n\r\n**input :** Graph G\r\n\r\n...,<java><algorithm><graph-algorithm>,I am implementing a Minimum Spanning Forrest a...,2
1,47303682,"While converting the data frame to HTML, Date ...",<html><r><dataframe>,"R Studio: Date is getting converted to number,...",2
2,46431243,"<p>This should be an easy one, but somehow I c...",<python><pandas><dataframe><pandas-groupby>,Pandas - dataframe groupby - how to get sum of...,0
3,45306324,<p>Let say I have a component in Angular 2 tha...,<angular><typescript><rxjs>,Casting results from Observable.forkJoin to th...,0
4,49479547,<p>I'm setting a var using \n<code>set TEST_VA...,<c><windows><batch-file>,Environment variable set in batch file cannot ...,1


In [ ]:
DevNull = TrainDF["target"].value_counts().plot.pie(subplots = True, legend = True, autopct = '%2.3f%%', title = "Отношение классов")

# Preprocessing data

## Removing punctuation and extra spaces

In [18]:
TestDF["Text"] = TestDF.Body + " " + TestDF.Tags + " " + TestDF.Title
TestDF["Text"] = TestDF["Text"].apply(lambda text: re.sub(r'[^(a-zA-Z)\s]', ' ', text))
TestDF["Text"] = TestDF["Text"].apply(lambda text: re.sub('^\s+|\n|\r|\s+$', ' ', text))
TestDF["Text"] = TestDF["Text"].apply(lambda text: re.sub('\ {2,}', ' ', text))
TestDF["Text"][0]

'I was given a decision tree with sample data in class to solve After computing the gaining splitting tree with the sample data provided I ended up with the same tree that was in the question What does that mean Can someone explain what happened there math machine learning artificial intelligence decision tree entropy Desecion trees ended up with same given tree after gain split computation '

In [19]:
TrainDF["Text"] = TrainDF.Body + " " + TrainDF.Tags + " " + TrainDF.Title
TrainDF["Text"] = TrainDF["Text"].apply(lambda text: re.sub(r'[^(a-zA-Z)\s]', ' ', text))
TrainDF["Text"] = TrainDF["Text"].apply(lambda text: re.sub('^\s+|\n|\r|\s+$', ' ', text))
TrainDF["Text"] = TrainDF["Text"].apply(lambda text: re.sub('\ {2,}', ' ', text))
TrainDF["Text"][0]

'Algorithmm input Graph G output Set of MSTs T begin T null E G Edges for all vertices in G Create a tree t having single vertex b add t to T end for repeat Find an edge e E having minimum weight such that one end belongs to t T and the other end does not belongs to any of the trees in T Add e to t until e NULL I m stuck on the logic for the highlighted block I ve used simple objects for vertex edge and tree And for their sets used array of Objects java algorithm graph algorithm I am implementing a Minimum Spanning Forrest algorihm in java But stuck on how to write a loop'

## Checking for invalid values (NA and Null)

In [20]:
print("TestDF:")
print(" ID:")
print("  NA = " + str(TestDF["Id"].isna().sum()))
print("  Null = " + str(TestDF["Id"].isnull().sum()))

TestDF:
 ID:
  NA = 0
  Null = 0


In [21]:
print("TrainDF:")
print(" ID:")
print("  NA = " + str(TrainDF["Id"].isna().sum()))
print("  Null = " + str(TrainDF["Id"].isnull().sum()))
print(" target:")
print("  NA = " + str(TrainDF["target"].isna().sum()))
print("  Null = " + str(TrainDF["target"].isnull().sum()))

TrainDF:
 ID:
  NA = 0
  Null = 0
 target:
  NA = 0
  Null = 0


No invalid values :)

## Tokenizing

In [22]:
Tokenizer = WordPunctTokenizer()
TestDF["Text"] = TestDF["Text"].apply(lambda text: Tokenizer.tokenize(text.lower()))
TestDF["Text"].head()

0    [i, was, given, a, decision, tree, with, sampl...
1    [p, i, am, moving, a, test, application, over,...
2    [p, i, would, like, to, be, able, to, perform,...
3    [p, i, m, using, a, href, https, github, com, ...
4    [i, have, some, test, cases, and, i, want, the...
Name: Text, dtype: object

In [23]:
TrainDF["Text"] = TrainDF["Text"].apply(lambda text: Tokenizer.tokenize(text.lower()))
TrainDF["Text"].head()

0    [algorithmm, input, graph, g, output, set, of,...
1    [while, converting, the, data, frame, to, html...
2    [p, this, should, be, an, easy, one, but, some...
3    [p, let, say, i, have, a, component, in, angul...
4    [p, i, m, setting, a, var, using, code, set, t...
Name: Text, dtype: object

## Counting and removing stop words

In [24]:
# Length of word must be greater than "2". No explanation, just intuition :)
def GetStopwords(text):
    StopWords = []
    for w in text:
        if (w in stopwords.words("english")) or (len(w) <= 2):
            StopWords.append(w)
    return StopWords

def RemoveStopwords(text):
    PureWords = []
    for w in text:
        if w not in stopwords.words("english") and len(w) > 2:
            PureWords.append(w)
    return PureWords

In [25]:
TestDF["Stop"] = TestDF["Text"].apply(GetStopwords)
TestDF["NumOfStopwords"] = TestDF["Stop"].apply(lambda text: len(text))
TestDF["Text"] = TestDF["Text"].apply(RemoveStopwords)
TestDF.head()

KeyboardInterrupt: ignored

In [ ]:
TrainDF["Stop"] = TrainDF["Text"].apply(GetStopwords)
TrainDF["NumOfStopwords"] = TrainDF["Stop"].apply(lambda text: len(text))
TrainDF["Text"] = TrainDF["Text"].apply(RemoveStopwords)
TrainDF.tail()

## Lemmatization

In [30]:
def Lemmatize(text):
    Lemmatizer = WordNetLemmatizer()
    Words = []
    for w in text:
        Words.append(Lemmatizer.lemmatize(w))
    return Words

TestDF["Text"] = TestDF["Text"].apply(Lemmatize)
TrainDF["Text"] = TrainDF["Text"].apply(Lemmatize)

# Extra features

In [31]:
# Number of tags
TestDF["NumOfTags"] = TestDF["Tags"].apply(lambda text: len(re.findall('<', text)))
TrainDF["NumOfTags"] = TrainDF["Tags"].apply(lambda text: len(re.findall('<', text)))

In [32]:
# Number of words
TestDF["NumOfWords"] = TestDF["Text"].apply(lambda text: len(text))
TrainDF["NumOfWords"] = TrainDF["Text"].apply(lambda text: len(text))

In [33]:
# Number of letters
def NumberOfLetters(text):
    counter = 0
    for w in text:
        counter += len(w)
    return counter

TestDF["NumOfLetters"] = TestDF["Text"].apply(NumberOfLetters)
TrainDF["NumOfLetters"] = TrainDF["Text"].apply(NumberOfLetters)

In [34]:
SaveParquet(TestDF, Path + "test.preprocessed.parquet")
SaveParquet(TrainDF, Path + "train.preprocessed.parquet")

# One Hot Encoding

In [35]:
# Load preprocessed data frames
# TestDF = ReadParquet(Path + "test.preprocessed.parquet")
# TrainDF = ReadParquet(Path + "train.preprocessed.parquet")

In [36]:
# Я либо чего-то в этой жизни не понимаю, 
# либо тот, кто придумал OHE, употреблял что-то крайне тяжёлое.
# Алгоритм крайне затратный в плане ресурсов ПК.
# Вот зачем было для каждого токена делать вектор, в котором 
# тупо ставить единицу по индексу, равному индексу этого токена в словаре?
# Если с таким же успехом можно просто оставлять индекс из этого словаря вместо вектора.
# Я, если честно, понимая, насколько огромным получится словарь, 
# не стал насиловать свои 8Gb RAM и 12Gb RAM Collab'а.

# Word2Vec

In [43]:
# Load preprocessed data frames
from gensim.models import KeyedVectors
TestDF = ReadParquet(Path + "test.preprocessed.parquet")
TrainDF = ReadParquet(Path + "train.preprocessed.parquet")
W2V_OurTrain = KeyedVectors.load_word2vec_format(Path + 'W2V_OurTrain.bin', binary = True)
W2V_OurTest = KeyedVectors.load_word2vec_format(Path + 'W2V_OurTest.bin', binary = True)

In [38]:
W2V_OurTest = Word2Vec(TestDF["Text"].values, min_count = 1, size = 100, window = 5)
W2V_OurTrain = Word2Vec(TrainDF["Text"].values, min_count = 1, size = 100, window = 5)

In [39]:
W2V_OurTest.wv.save_word2vec_format(Path + 'W2V_OurTest.bin', binary = True)
W2V_OurTrain.wv.save_word2vec_format(Path + 'W2V_OurTrain.bin', binary = True)

In [ ]:
words = sorted(W2V_OurTest.wv.vocab.keys(), 
               key = lambda word: W2V_OurTest.wv.vocab[word].count, 
               reverse = True)
words[:10]

In [ ]:
words = sorted(W2V_OurTrain.wv.vocab.keys(), 
               key = lambda word: W2V_OurTrain.wv.vocab[word].count, 
               reverse = True)
words[:10]

In [ ]:
import numpy as np
word_vectors = np.array([W2V_OurTrain.wv.get_vector(vector) for vector in words])
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words),100)
assert np.isfinite(word_vectors).all()

In [ ]:
test_vector = []
for i in range(len(TestDF['Text'])):
  vector = 0
  for w in TestDF['Text'][i]:
    try: 
        vector += W2V_OurTest.wv[w]
    except:
        continue
  test_vector.append(vector)
test_vector

In [48]:
len(test_vector)

12000

In [ ]:
train_vector = []
for i in range(len(TrainDF['Text'])):
  vector = 0
  for w in TrainDF['Text'][i]:
    try: 
        vector += W2V_OurTrain.wv[w]
    except:
        continue
  train_vector.append(vector)
train_vector

In [62]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
y = TrainDF['target']
X_train, X_test, y_train, y_test = train_test_split(train_vector, y, test_size=0.2, random_state=666)


In [ ]:
from sklearn import svm
from sklearn.metrics import recall_score

C = 1.0  # = self._alpha in our algorithm
model = svm.SVC(kernel='linear', C=C)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
print(recall_score(y_test, y_predict, average=None))

In [ ]:
knn = KNeighborsClassifier().fit(X_train, y_train)
acc_knn = np.mean(knn.predict(X_train) == y_train)
test_acc_knn = np.mean(knn.predict(X_test) == y_test)
print(f"Train Accuracy: {acc_knn}, Test Accuracy: {test_acc_knn}")

In [ ]:
nb = GaussianNB().fit(X_train, y_train)
acc_nb = np.mean(nb.predict(X_train) == y_train)
test_acc_nb = np.mean(nb.predict(X_test) == y_test)
print(f"Train Accuracy: {acc_nb}, Test Accuracy: {test_acc_nb}")

In [ ]:
gs = GridSearchCV(knn, param_grid={"n_neighbors": range(1,20)})
gs.fit(X_train, y_train)
#gs.best_params_, gs.best_score_, gs.best_estimator_.n_neighbors
best = gs.best_estimator_.n_neighbors
print(f"Grid Search: {gs.best_params_, gs.best_score_, gs.best_estimator_.n_neighbors}")

In [ ]:
knn_new = KNeighborsClassifier(n_neighbors = best).fit(train_vecX_traintor, y_train)
acc_knn_new = np.mean(knn_new.predict(X_train) == y_train)
test_acc_knn_new = np.mean(knn_new.predict(X_test) == y_test)
print(f"Train Accuracy: {acc_knn_new}, Test Accuracy: {test_acc_knn_new}")